# NLP Kaggle Tutorial (Phase 2)


The [actual tutorial](https://www.kaggle.com/c/word2vec-nlp-tutorial#part-1-for-beginners-bag-of-words) was developed by **Angela Chapman** during her summer 2014 internship at Kaggle.  
Full credits to her!

This notebook by Prashant Brahmbhatt                      

### Introducing Distributed Word Vectors

In [1]:
import re
import nltk
import logging
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from nltk.corpus import stopwords

**Word2Vec** does not need labels in order to create meaningful representations.  
This is useful, since most data in the real world is unlabeled. If the network is given enough training data (tens of billions of words), it produces word vectors with intriguing characteristics.  
Words with similar meanings appear in clusters, and clusters are spaced such that some word relationships, such as analogies, can be reproduced using vector math.   
**Example**  
"king - man + woman = queen."

______________________

Check out [Google's code, writeup, and the accompanying papers](https://code.google.com/archive/p/word2vec/). [This presentation](https://docs.google.com/file/d/0B7XkCwpI5KDYRWRnd1RzWXQ2TWc/edit) is also helpful. The original code is in C, but it has since been ported to other languages, including Python.  
Recent work out of Stanford has also applied deep learning to sentiment analysis; their code is available in Java. However, their approach, which relies on sentence parsing, cannot be applied in a straightforward way to paragraphs of arbitrary length.

__________________

## Preparing to Train the model

First, we read in the data with pandas, we now use data with 50,000 additional reviews with no labels. 
When we built the **Bag of Words** model previously extra unlabeled training reviews were not useful.  
However, since **Word2Vec** can learn from unlabeled data, these extra 50,000 reviews can now be used.

In [2]:
train = pd.read_csv('kaggleNLP/labeledTrainData.tsv',sep = '\t' , quoting = 3)
test = pd.read_csv('kaggleNLP/testData.tsv', sep = '\t', quoting = 3)
unlabeled = pd.read_csv('kaggleNLP/unlabeledTrainData.tsv', sep = '\t', quoting = 3 )

In [3]:
print("Labeled Train reviews {}\nLabeled Test reviews {}\nUnlabeled reviews {}".
      format(train['review'].size, test['review'].size, unlabeled['review'].size))

Labeled Train reviews 25000
Labeled Test reviews 25000
Unlabeled reviews 50000


First, to train **Word2Vec** it is better not to remove **stop words** because the algorithm relies on the broader context of the sentence in order to produce high-quality word vectors.  
For this reason, we will make stop word removal optional in the functions below. It also might be better not to remove numbers, but its up to the programmer.

In [4]:
def review_to_wordlist(review, remove_stopwords = False):
    # Function to convert a document to a sequence of words
    
    #Removing HTML
    review_text = BeautifulSoup(review).get_text()
    
    #Removing Non Letters
    review_text = re.sub('[^a-zA-Z]',' ',review_text)
    
    #Convert words to lowercase and split
    words = review_text.lower().split()
    
    #Optionally removing stopwords
    if remove_stopwords:
        words = [w for w in words if w not in set(stopwords.words('english'))]
    #Returing the list of words
    return (words)

We want a specific input format. **Word2Vec** expects single sentences, each one as a list of words. In other words, the input format is a **list of lists**.

Spltting the paragraph is not very strightforward. There are all kinds of gotchas in natural language. English sentences can end with **"?"**, **"!"**, **"""**, or **"."**, among other things, and spacing and capitalization are not reliable guides either.  
For this reason, we use **NLTK's punkt tokenizer** for sentence splitting.

_____

In [5]:
#nltk.download()
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

_______

Defining a function to split a review into parsed sentences

In [6]:
def review_to_sentences(review , tokenizer, remove_stopwords = False):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words

    # Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())

    #Looping over each sentence
    sentences = []

    #If sentence is empty skip otherwie call review_to_wordlist
    for sentence in raw_sentences:
        if len(sentence) >0:
            sentences.append(review_to_wordlist(sentence, remove_stopwords))
        
    return sentences

Now we can apply this function to prepare our data for input to **Word2Vec**

In [7]:
sentences = []
print('Parsing sentences from Reviews')
for rev in train['review']:
    sentences += review_to_sentences(rev, tokenizer)
    
print('Parsing sentences for unlabeled data')
for rev in unlabeled['review']:
    sentences += review_to_sentences(rev, tokenizer)

Parsing sentences from Reviews


C:\Users\prash\Anaconda3\envs\tensorflow\lib\site-packages\bs4\__init__.py:273: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\Users\prash\Anaconda3\envs\tensorflow\lib\site-packages\bs4\__init__.py:273: UserWarning: "b'...'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\Users\prash\Anaconda3\envs\tensorflow\lib\site-packages\bs4\__init__.py:336: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Parsing sentences for unlabeled data


C:\Users\prash\Anaconda3\envs\tensorflow\lib\site-packages\bs4\__init__.py:336: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\prash\Anaconda3\envs\tensorflow\lib\site-packages\bs4\__init__.py:336: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\prash\Anaconda3\envs\tensorflow\lib\site-packages\bs4\__init__.py:273: UserWarning: "b'... ...'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.


We can see we have got warnings which are about the URLs in the sentences which are not a big trouble. Although we may want to remove these too.

___________

**Unconfirmed** (don't Read)

A minor detail to note is the difference between the **"+="** and **"append"** when it comes to Python lists. 
In many applications the two are interchangeable, but here they are not. 
When appending a list of lists to another list of lists, **"append"** will only append the first list; you need to use **"+="** in order to join all of the lists at once.

_____

## Training and Saving the Model

There are a number of parameter choices that affect the run time and the quality of the final model that is produced.  
For details on the algorithms below, see the word2vec [API documentation](radimrehurek.com/gensim/models/word2vec.html) as well as the [Google documentation](https://code.google.com/archive/p/word2vec/).

- **Architecture**: Architecture options are skip-gram (default) or continuous bag of words. Here it was  found that skip-gram was very slightly slower but produced better results.


- **Training algorithm**: Hierarchical softmax (default) or negative sampling. Here, the default worked well. 


- **Downsampling of frequent words**: The Google documentation recommends values between .00001 and .001. Here values closer 0.001 seemed to improve the accuracy of the final model.


- **Word vector dimensionality**: More features result in longer runtimes, and often, but not always, result in better models. Reasonable values can be in the tens to hundreds; we used 300.


- **Context / window size**: How many words of context should the training algorithm take into account? 10 seems to work well for hierarchical softmax (more is better, up to a point).


- **Worker threads**: Number of parallel processes to run. This is computer-specific, but between 4 and 6 should work on most systems.


- **Minimum word count**: This helps limit the size of the vocabulary to meaningful words. Any word that does not occur at least this many times across all documents is ignored. Reasonable values could be between 10 and 100. In this case, since each movie occurs 30 times, we set the minimum word count to 40, to avoid attaching too much importance to individual movie titles. This resulted in an overall vocabulary size of around 15,000 words. Higher values also help limit run time.


In [17]:
# Using the built-in logging module and configuring it so that Word2Vec 
# creates nice output messages
logging.basicConfig(format = '%(asctime)s: %(levelname)s: %(messages)s',level = logging.INFO)

#Setting values for various parameters
num_features = 300    #Word vector dimesionality
min_word_count = 40   #Minimum Word Count
num_workers = 4       #Number of threads to Run in parallel
context = 10          #Context window size
downsampling = 1e-3   #Downsampling setting for frequent words

#Initializing and Training the model
from gensim.models import word2vec
print('Training Model')
model = word2vec.Word2Vec(sentences,
                 workers = num_workers,
                 size = num_features,
                 min_count = min_word_count,
                 window = context,
                 sample = downsampling)

#If we do not plan to train the model further calling init_sims makes the model more memory efficient
model.init_sims(replace = True)

# It can be helpful to create a meaningful model name and save the model for later use. 
# We can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

Training Model


--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\logging\__init__.py", line 986, in emit
    msg = self.format(record)
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\logging\__init__.py", line 836, in format
    return fmt.format(record)
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\logging\__init__.py", line 576, in format
    s = self.formatMessage(record)
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\logging\__init__.py", line 545, in formatMessage
    return self._style.format(record)
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\logging\__init__.py", line 389, in format
    return self._fmt % record.__dict__
KeyError: 'messages'
Call stack:
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\pras

--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\logging\__init__.py", line 986, in emit
    msg = self.format(record)
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\logging\__init__.py", line 836, in format
    return fmt.format(record)
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\logging\__init__.py", line 576, in format
    s = self.formatMessage(record)
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\logging\__init__.py", line 545, in formatMessage
    return self._style.format(record)
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\logging\__init__.py", line 389, in format
    return self._fmt % record.__dict__
KeyError: 'messages'
Call stack:
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\pras

--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\logging\__init__.py", line 986, in emit
    msg = self.format(record)
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\logging\__init__.py", line 836, in format
    return fmt.format(record)
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\logging\__init__.py", line 576, in format
    s = self.formatMessage(record)
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\logging\__init__.py", line 545, in formatMessage
    return self._style.format(record)
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\logging\__init__.py", line 389, in format
    return self._fmt % record.__dict__
KeyError: 'messages'
Call stack:
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\pras

--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\logging\__init__.py", line 986, in emit
    msg = self.format(record)
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\logging\__init__.py", line 836, in format
    return fmt.format(record)
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\logging\__init__.py", line 576, in format
    s = self.formatMessage(record)
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\logging\__init__.py", line 545, in formatMessage
    return self._style.format(record)
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\logging\__init__.py", line 389, in format
    return self._fmt % record.__dict__
KeyError: 'messages'
Call stack:
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\pras

...

--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\logging\__init__.py", line 986, in emit
    msg = self.format(record)
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\logging\__init__.py", line 836, in format
    return fmt.format(record)
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\logging\__init__.py", line 576, in format
    s = self.formatMessage(record)
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\logging\__init__.py", line 545, in formatMessage
    return self._style.format(record)
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\logging\__init__.py", line 389, in format
    return self._fmt % record.__dict__
KeyError: 'messages'
Call stack:
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\pras

--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\logging\__init__.py", line 986, in emit
    msg = self.format(record)
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\logging\__init__.py", line 836, in format
    return fmt.format(record)
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\logging\__init__.py", line 576, in format
    s = self.formatMessage(record)
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\logging\__init__.py", line 545, in formatMessage
    return self._style.format(record)
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\logging\__init__.py", line 389, in format
    return self._fmt % record.__dict__
KeyError: 'messages'
Call stack:
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\pras

--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\logging\__init__.py", line 986, in emit
    msg = self.format(record)
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\logging\__init__.py", line 836, in format
    return fmt.format(record)
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\logging\__init__.py", line 576, in format
    s = self.formatMessage(record)
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\logging\__init__.py", line 545, in formatMessage
    return self._style.format(record)
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\logging\__init__.py", line 389, in format
    return self._fmt % record.__dict__
KeyError: 'messages'
Call stack:
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\pras

--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\site-packages\gensim\utils.py", line 688, in save
    _pickle.dump(self, fname_or_handle, protocol=pickle_protocol)
TypeError: file must have a 'write' attribute

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\logging\__init__.py", line 986, in emit
    msg = self.format(record)
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\logging\__init__.py", line 836, in format
    return fmt.format(record)
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\logging\__init__.py", line 576, in format
    s = self.formatMessage(record)
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\logging\__init__.py", line 545, in formatMessage
    return self._style.format(record)
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\logging\__init__.py", line 389, in format
    return

--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\site-packages\gensim\utils.py", line 688, in save
    _pickle.dump(self, fname_or_handle, protocol=pickle_protocol)
TypeError: file must have a 'write' attribute

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\logging\__init__.py", line 986, in emit
    msg = self.format(record)
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\logging\__init__.py", line 836, in format
    return fmt.format(record)
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\logging\__init__.py", line 576, in format
    s = self.formatMessage(record)
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\logging\__init__.py", line 545, in formatMessage
    return self._style.format(record)
  File "C:\Users\prash\Anaconda3\envs\tensorflow\lib\logging\__init__.py", line 389, in format
    return

## Exploring the Model Results

In [18]:
model.doesnt_match("man woman child kitchen".split())

C:\Users\prash\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.


'kitchen'

In [30]:
model.doesnt_match("france england berlin india".split())

C:\Users\prash\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.


'berlin'

In [23]:
model.doesnt_match("paris berlin london austria".split())

C:\Users\prash\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.


'paris'

In [24]:
model.most_similar("man")

C:\Users\prash\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('woman', 0.6203411817550659),
 ('lady', 0.5872369408607483),
 ('lad', 0.5774174928665161),
 ('soldier', 0.5325820446014404),
 ('millionaire', 0.5276577472686768),
 ('men', 0.5220717191696167),
 ('monk', 0.5197499990463257),
 ('businessman', 0.505675196647644),
 ('guy', 0.5039936900138855),
 ('farmer', 0.49234679341316223)]

In [25]:
model.most_similar("woman")

C:\Users\prash\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('girl', 0.6871989965438843),
 ('prostitute', 0.6852065324783325),
 ('lady', 0.6830874681472778),
 ('widow', 0.6444469690322876),
 ('man', 0.6203412413597107),
 ('waitress', 0.6119025945663452),
 ('housewife', 0.5968441367149353),
 ('nun', 0.5800549983978271),
 ('heiress', 0.5719454884529114),
 ('nurse', 0.5618314743041992)]

In [35]:
model.most_similar("avengers")

C:\Users\prash\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('hercules', 0.49119964241981506),
 ('ark', 0.4841819703578949),
 ('hobbit', 0.4836318790912628),
 ('smallville', 0.4830237627029419),
 ('gigli', 0.47936171293258667),
 ('showgirls', 0.46363702416419983),
 ('spongebob', 0.4632878303527832),
 ('leprechaun', 0.46312251687049866),
 ('anchorman', 0.4606848359107971),
 ('luther', 0.4588777422904968)]